# **NeurIPS Targets ChemBerta**

In [1]:
import random
import numpy as np
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModel, RobertaTokenizer, RobertaModel
transformers.logging.set_verbosity_error()
import re
import tensorflow as tf
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)

2025-06-28 16:12:58.057755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751127178.328210      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751127178.406472      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# 1. Load the dataset
data0 = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
display(data0.info())
targets = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
data = data0.drop('id',axis=1)
data.columns=['text']+targets
display(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7973 entries, 0 to 7972
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       7973 non-null   int64  
 1   SMILES   7973 non-null   object 
 2   Tg       511 non-null    float64
 3   FFV      7030 non-null   float64
 4   Tc       737 non-null    float64
 5   Density  613 non-null    float64
 6   Rg       614 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 436.2+ KB


None

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,text,Tg,FFV,Tc,Density,Rg
0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN
1,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN
3,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN
4,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN
...,...,...,...,...,...,...
7968,*Oc1cc(CCCCCCCC)cc(OC(=O)c2cccc(C(*)=O)c2)c1,NaN,0.367498,NaN,NaN,NaN
7969,*C(=O)OCCN(CCOC(=O)c1ccc2c(c1)C(=O)N(c1cccc(N3...,NaN,0.353280,NaN,NaN,NaN
7970,*c1cc(C(=O)NCCCCCCCC)cc(N2C(=O)c3ccc(-c4ccc5c(...,NaN,0.369411,NaN,NaN,NaN
7971,*C=C(*)c1ccccc1C,261.662355,NaN,NaN,NaN,NaN


In [3]:
TEST0 = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
TEST = TEST0[['SMILES']]
TEST.columns=['text']

In [4]:
# 2. Tokenization and data preparation
model_path="/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM"
tokenizer = AutoTokenizer.from_pretrained(model_path)
max_len = 128

def create_data(text):
    encoded = tokenizer.batch_encode_plus(
        text.tolist(),
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True
    )

    input_ids = np.array(encoded["input_ids"], dtype="int32")
    attention_masks = np.array(encoded["attention_mask"], dtype="float64")

    return {"input_ids": input_ids,"attention_mask": attention_masks}

In [5]:
from transformers import AutoModel, AutoTokenizer
import torch

def extract_roberta_features(texts, model_path, batch_size=32, max_len=512):
    """
    Extract features from text using RoBERTa model

    Args:
        texts: List of input text strings
        model_path: Path to model (local or Hugging Face hub name)
        batch_size: Batch size for processing
        max_len: Maximum sequence length

    Returns:
        numpy array of extracted features
    """
    # Load model and tokenizer
    model = AutoModel.from_pretrained(model_path, local_files_only=True)
    tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    features = []

    # Process in batches
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        # Tokenization (using PyTorch tensors)
        encoded = tokenizer(
            batch_texts,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        # Move tensors to the same device as model
        encoded = {k: v.to(device) for k, v in encoded.items()}

        # Extract features
        with torch.no_grad():
            outputs = model(**encoded)

        # Average pooling (mean of last hidden states)
        pooled_features = outputs.last_hidden_state.mean(dim=1)

        # Move to CPU and convert to numpy
        features.append(pooled_features.cpu().numpy())

    return np.vstack(features)

In [6]:
# Learning rate scheduler
def scheduler(epoch):
    learning_rate = 2e-5
    if epoch == 0:
        return learning_rate * 0.1
    else:
        return learning_rate * (0.95 ** epoch)

In [7]:
# 4. Train the model (modified for feature extraction approach)
def train_model(train_texts, train_targets):
    print("Extracting features from training data...")
    train_features = extract_roberta_features(train_texts, model_path)

    model = XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist",
        verbosity=0
    )

    model.fit(train_features, train_targets)
    return model

In [8]:
# 5. Predict and evaluate (for feature extraction approach)
def evaluate_model(model, test_targets):
    # Extract features from test data
    print("Extracting features from test data...")
    test_texts = test['text'].tolist()
    test_features = extract_roberta_features(test_texts, model_path)

    # Make predictions
    predictions = model.predict(test_features)
    predictions = predictions.flatten()

    # Evaluation metrics
    mse = np.mean((predictions - test_targets) ** 2)
    mae = np.mean(np.abs(predictions - test_targets))
    rmse = np.sqrt(mse)

    # Correlation coefficient
    correlation = np.corrcoef(predictions, test_targets)[0, 1]

    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"Correlation: {correlation:.4f}")

    return predictions

In [9]:
def build_regression_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='mse',
                  metrics=['mae'])

    return model

def train_nn_model(train_texts, train_targets):
    print("Extracting features for NN...")
    train_features = extract_roberta_features(train_texts, model_path)

    model = build_regression_model(train_features.shape[1])
    model.fit(
        train_features, train_targets,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=0
    )

    return model


In [10]:

if __name__ == "__main__":
    nn_models = {}
    xgb_models = {}

    for target in targets:
        filtered_df = data.dropna(subset=[target])
        train, test = train_test_split(filtered_df, test_size=0.1, random_state=42)

        train_texts = train['text'].tolist()
        test_texts = test['text'].tolist()
        train_targets = train[target].values
        test_targets = test[target].values

        print(f"\nTraining for target: {target}")

        if target in ['Tg', 'Rg']:
            model = train_nn_model(train_texts, train_targets)
            nn_models[target] = model
        elif target in ['Tc', 'FFV']:
            model = train_model(train_texts, train_targets)
            xgb_models[target] = model
        elif target == 'Density':
            nn_model = train_nn_model(train_texts, train_targets)
            xgb_model = train_model(train_texts, train_targets)
            nn_models[target] = nn_model
            xgb_models[target] = xgb_model

        print(f"Evaluating for target: {target}")
        test_features = extract_roberta_features(test_texts, model_path)

        if target in ['Tg', 'Rg']:
            predictions = nn_models[target].predict(test_features).flatten()
        elif target in ['Tc', 'FFV']:
            predictions = xgb_models[target].predict(test_features)
        elif target == 'Density':
            nn_pred = nn_models[target].predict(test_features).flatten()
            xgb_pred = xgb_models[target].predict(test_features)
            predictions = 0.3 * nn_pred + 0.7 * xgb_pred

        mse = np.mean((predictions - test_targets) ** 2)
        mae = np.mean(np.abs(predictions - test_targets))
        rmse = np.sqrt(mse)
        correlation = np.corrcoef(predictions, test_targets)[0, 1]

        print(f"MSE: {mse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"Correlation: {correlation:.4f}")



Training for target: Tg
Extracting features for NN...


2025-06-28 16:13:36.394259: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Evaluating for target: Tg
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
MSE: 3311.9216
MAE: 45.1261
RMSE: 57.5493
Correlation: 0.8192

Training for target: FFV
Extracting features from training data...
Evaluating for target: FFV
MSE: 0.0003
MAE: 0.0085
RMSE: 0.0185
Correlation: 0.8258

Training for target: Tc
Extracting features from training data...
Evaluating for target: Tc
MSE: 0.0025
MAE: 0.0320
RMSE: 0.0495
Correlation: 0.8511

Training for target: Density
Extracting features for NN...
Extracting features from training data...
Evaluating for target: Density
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
MSE: 0.0140
MAE: 0.0645
RMSE: 0.1185
Correlation: 0.5064

Training for target: Rg
Extracting features for NN...
Evaluating for target: Rg
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
MSE: 6.4795
MAE: 1.8262
RMSE: 2.5455
Correlation: 0.8392


In [11]:

test_texts = TEST['text'].tolist()
test_features = extract_roberta_features(test_texts, model_path)

RESULT = np.zeros((len(TEST), len(targets)))

for i, target in enumerate(targets):
    if target in ['Tg', 'Rg']:
        pred = nn_models[target].predict(test_features).flatten()
    elif target in ['Tc', 'FFV']:
        pred = xgb_models[target].predict(test_features)
    elif target == 'Density':
        nn_pred = nn_models[target].predict(test_features).flatten()
        xgb_pred = xgb_models[target].predict(test_features)
        pred = 0.3 * nn_pred + 0.7 * xgb_pred

    RESULT[:, i] = pred

submit = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')
submit.iloc[:, 1:] = RESULT
submit.to_csv('submission.csv', index=False)
display(submit)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


/tmp/ipykernel_13/444233199.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[124.49169159 168.24113464 112.41947174]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submit.iloc[:, 1:] = RESULT
/tmp/ipykernel_13/444233199.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.37532592 0.3730531  0.35220748]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submit.iloc[:, 1:] = RESULT
/tmp/ipykernel_13/444233199.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.21702242 0.23187959 0.25791746]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submit.iloc[:, 1:] = RESULT
/tmp/ipykernel_13/444233199.py:19: FutureWarning: Setting an item of incompa

,id,Tg,FFV,Tc,Density,Rg
0,1109053969,124.491692,0.375326,0.217022,1.124340,20.228838
1,1422188626,168.241135,0.373053,0.231880,1.106312,22.918423
2,2032016830,112.419472,0.352207,0.257917,1.107689,19.364748
